In [3]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pprint
import pandas as pd
import mcfg
import json
import warnings
from selenium import webdriver
import numpy as np
from sqlalchemy import create_engine

In [3]:
# Парсинг с МОС.РУ данных о всех клиниках (всего 306 страниц на сайте с информацией о клиниках - проходим по каждой)
k = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
for i in range(1, 307):
    url = f'https://www.mos.ru/clinics/?page={i}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    clinics = soup.find_all('div', class_='css-qxz6vf-Box')
    time.sleep(random.uniform(1, 3))
    k.append(clinics)
    print(f'{i} ОБРАБОТАНО')


1 ОБРАБОТАНО
2 ОБРАБОТАНО
3 ОБРАБОТАНО
4 ОБРАБОТАНО
5 ОБРАБОТАНО
6 ОБРАБОТАНО
7 ОБРАБОТАНО
8 ОБРАБОТАНО
9 ОБРАБОТАНО
10 ОБРАБОТАНО
11 ОБРАБОТАНО
12 ОБРАБОТАНО
13 ОБРАБОТАНО
14 ОБРАБОТАНО
15 ОБРАБОТАНО
16 ОБРАБОТАНО
17 ОБРАБОТАНО
18 ОБРАБОТАНО
19 ОБРАБОТАНО
20 ОБРАБОТАНО
21 ОБРАБОТАНО
22 ОБРАБОТАНО
23 ОБРАБОТАНО
24 ОБРАБОТАНО
25 ОБРАБОТАНО
26 ОБРАБОТАНО
27 ОБРАБОТАНО
28 ОБРАБОТАНО
29 ОБРАБОТАНО
30 ОБРАБОТАНО
31 ОБРАБОТАНО
32 ОБРАБОТАНО
33 ОБРАБОТАНО
34 ОБРАБОТАНО
35 ОБРАБОТАНО
36 ОБРАБОТАНО
37 ОБРАБОТАНО
38 ОБРАБОТАНО
39 ОБРАБОТАНО
40 ОБРАБОТАНО
41 ОБРАБОТАНО
42 ОБРАБОТАНО
43 ОБРАБОТАНО
44 ОБРАБОТАНО
45 ОБРАБОТАНО
46 ОБРАБОТАНО
47 ОБРАБОТАНО
48 ОБРАБОТАНО
49 ОБРАБОТАНО
50 ОБРАБОТАНО
51 ОБРАБОТАНО
52 ОБРАБОТАНО
53 ОБРАБОТАНО
54 ОБРАБОТАНО
55 ОБРАБОТАНО
56 ОБРАБОТАНО
57 ОБРАБОТАНО
58 ОБРАБОТАНО
59 ОБРАБОТАНО
60 ОБРАБОТАНО
61 ОБРАБОТАНО
62 ОБРАБОТАНО
63 ОБРАБОТАНО
64 ОБРАБОТАНО
65 ОБРАБОТАНО
66 ОБРАБОТАНО
67 ОБРАБОТАНО
68 ОБРАБОТАНО
69 ОБРАБОТАНО
70 ОБРАБОТАНО
71 ОБРАБОТАНО
72 ОБРАБОТАНО
7

In [21]:
pprint.pprint(len(k)) # Количество страниц

306


In [80]:
# Вытаскиваем основную информацию из списка k
# список имеет стркутуру [[клиника1, клиника2, ...], [клиника8, ...]]
# поэтому перебираем 2 циклами - по страницам и по карточкам на страницах

columns = ['CLINIC_NAME', 'CLINIC_TYPE', 'CLINIC_ADDRESS', 'CLINIC_PHONE_NUMBER']
clinic_data_list = []


for page in k:
    for clinic in page:
        clinic_type = clinic.find('span', class_='css-1w28k3i-Text').text
        clinic_address = clinic.find('div', class_='css-1efsrtw-Box').text
        clinic_name = clinic.find('h2', class_='css-1c0yqy5-Heading-Text').a.text
        try:
            clinic_number = clinic.find('div', class_='css-11kiilf-Box').text
        except:
            clinic_number = None
        
        clinic_data_list.append([clinic_name, clinic_type, clinic_address, clinic_number])



In [81]:
clinics_df = pd.DataFrame(clinic_data_list, columns=columns) # Собираем предварительный датафрейм

In [42]:
clinics_df['id'] = range(1, len(clinics_df) + 1) # Формируем уникальные id для каждой записи

In [13]:
clinics_df

,id,clinic_name,clinic_type,clinic_address,clinic_phone_number
0,1,Женская консультация «Внуково» Государственное...,Женская консультация,"г. Москва, внутригородская территория поселени...",+7 (495) 827-27-09
1,2,ЭПИКРИЗ Государственное бюджетное учреждение з...,Научно-практический центр,"г. Москва, улица Расковой, д.14",+7 (999) 999-99-99
2,3,Детский эндоскопический центр Государственного...,Детская городская больница,"г. Москва, Садовая-Кудринская улица, д.15, с.11",+7 (499) 766-71-44
3,4,Государственное бюджетное учреждение здравоохр...,Диагностический центр (КДЦ),"г. Москва, улица Грекова, д.12",+7 (495) 568-11-13
4,5,Государственное бюджетное учреждение здравоохр...,Кожно-венерологический диспансер,"г. Москва, улица Маршала Мерецкова, д.1",+7 (499) 194-70-24
...,...,...,...,...,...
1588,2137,Центр лечения воспалительных заболеваний кишеч...,Детский центр лечения по гастроэнтерологии,"г. Москва, 4-й Добрынинский переулок, д.1/9, с.21",+7 (495) 959-88-00
1589,2138,Центр амбулаторной медицинской помощи участник...,Диагностический центр (КДЦ),"г. Москва, Стартовая улица, д.4, с.3",+7 (495) 474-99-10
1590,2139,Центр психолого-психотерапевтической помощи,Психиатрическая клиническая больница,"г. Москва, Береговой проезд, д.8, с.2",None
1591,2140,Центр лечения целиакии Государственного бюджет...,Детский центр лечения по гастроэнтерологии,"г. Москва, улица Героев Панфиловцев, д.28",+7 (495) 539-24-73


In [14]:
clinics_df['clinic_coords'] = None # Формируем новый атрибут для дальнейшего заполнения координатами

In [15]:
clinics_df

,id,clinic_name,clinic_type,clinic_address,clinic_phone_number,clinic_coords
0,1,Женская консультация «Внуково» Государственное...,Женская консультация,"г. Москва, внутригородская территория поселени...",+7 (495) 827-27-09,None
1,2,ЭПИКРИЗ Государственное бюджетное учреждение з...,Научно-практический центр,"г. Москва, улица Расковой, д.14",+7 (999) 999-99-99,None
2,3,Детский эндоскопический центр Государственного...,Детская городская больница,"г. Москва, Садовая-Кудринская улица, д.15, с.11",+7 (499) 766-71-44,None
3,4,Государственное бюджетное учреждение здравоохр...,Диагностический центр (КДЦ),"г. Москва, улица Грекова, д.12",+7 (495) 568-11-13,None
4,5,Государственное бюджетное учреждение здравоохр...,Кожно-венерологический диспансер,"г. Москва, улица Маршала Мерецкова, д.1",+7 (499) 194-70-24,None
...,...,...,...,...,...,...
1588,2137,Центр лечения воспалительных заболеваний кишеч...,Детский центр лечения по гастроэнтерологии,"г. Москва, 4-й Добрынинский переулок, д.1/9, с.21",+7 (495) 959-88-00,None
1589,2138,Центр амбулаторной медицинской помощи участник...,Диагностический центр (КДЦ),"г. Москва, Стартовая улица, д.4, с.3",+7 (495) 474-99-10,None
1590,2139,Центр психолого-психотерапевтической помощи,Психиатрическая клиническая больница,"г. Москва, Береговой проезд, д.8, с.2",None,None
1591,2140,Центр лечения целиакии Государственного бюджет...,Детский центр лечения по гастроэнтерологии,"г. Москва, улица Героев Панфиловцев, д.28",+7 (495) 539-24-73,None


In [20]:
# С помощью API вытаскиваем координаты

counter = 0
for i in list(clinics_df['clinic_address']):
    counter +=1
    url = url = f"https://geocode-maps.yandex.ru/1.x/?apikey={mcfg.api_key()}&geocode={i}&format=json"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        geo_object = data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]
        coordinates = geo_object["Point"]["pos"]
        clinics_df.loc[clinics_df['clinic_address'] == i, 'clinic_coords'] = coordinates
    print(counter, str(response.status_code))

1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 200
20 200
21 200
22 200
23 200
24 200
25 200
26 200
27 200
28 200
29 200
30 200
31 200
32 200
33 200
34 200
35 200
36 200
37 200
38 200
39 200
40 200
41 200
42 200
43 200
44 200
45 200
46 200
47 200
48 200
49 200
50 200
51 200
52 200
53 200
54 200
55 200
56 200
57 200
58 200
59 200
60 200
61 200
62 200
63 200
64 200
65 200
66 200
67 200
68 200
69 200
70 200
71 200
72 200
73 200
74 200
75 200
76 200
77 200
78 200
79 200
80 200
81 200
82 200
83 200
84 200
85 200
86 200
87 200
88 200
89 200
90 200
91 200
92 200
93 200
94 200
95 200
96 200
97 200
98 200
99 200
100 200
101 200
102 200
103 200
104 200
105 200
106 200
107 200
108 200
109 200
110 200
111 200
112 200
113 200
114 200
115 200
116 200
117 200
118 200
119 200
120 200
121 200
122 200
123 200
124 200
125 200
126 200
127 200
128 200
129 200
130 200
131 200
132 200
133 200
134 200
135 200
136 200
137 200
138 200
139 

In [21]:
clinics_df # Конечный датафрейм

,id,clinic_name,clinic_type,clinic_address,clinic_phone_number,clinic_coords
0,1,Женская консультация «Внуково» Государственное...,Женская консультация,"г. Москва, внутригородская территория поселени...",+7 (495) 827-27-09,37.313223 55.624652
1,2,ЭПИКРИЗ Государственное бюджетное учреждение з...,Научно-практический центр,"г. Москва, улица Расковой, д.14",+7 (999) 999-99-99,37.57245 55.786252
2,3,Детский эндоскопический центр Государственного...,Детская городская больница,"г. Москва, Садовая-Кудринская улица, д.15, с.11",+7 (499) 766-71-44,37.585503 55.766055
3,4,Государственное бюджетное учреждение здравоохр...,Диагностический центр (КДЦ),"г. Москва, улица Грекова, д.12",+7 (495) 568-11-13,37.66291 55.889316
4,5,Государственное бюджетное учреждение здравоохр...,Кожно-венерологический диспансер,"г. Москва, улица Маршала Мерецкова, д.1",+7 (499) 194-70-24,37.490587 55.792994
...,...,...,...,...,...,...
1588,2137,Центр лечения воспалительных заболеваний кишеч...,Детский центр лечения по гастроэнтерологии,"г. Москва, 4-й Добрынинский переулок, д.1/9, с.21",+7 (495) 959-88-00,37.6198 55.724824
1589,2138,Центр амбулаторной медицинской помощи участник...,Диагностический центр (КДЦ),"г. Москва, Стартовая улица, д.4, с.3",+7 (495) 474-99-10,37.703945 55.887372
1590,2139,Центр психолого-психотерапевтической помощи,Психиатрическая клиническая больница,"г. Москва, Береговой проезд, д.8, с.2",None,37.503962 55.755029
1591,2140,Центр лечения целиакии Государственного бюджет...,Детский центр лечения по гастроэнтерологии,"г. Москва, улица Героев Панфиловцев, д.28",+7 (495) 539-24-73,37.403944 55.851798


In [23]:
# Параметры подключения к БД
# Создаем подключение и сразу закрываем его

db_host = "51.250.99.11"
db_port = "5432"
db_name = "postgres"
db_user = "team18"
schema_name = 'team18'
db_password = "team_18"
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
engine.dispose()

In [25]:
clinics_df.to_sql('clinics', engine, schema=schema_name, if_exists='replace', index=False) # Загружаем датафрейм в БД

593

# Заметки

В датафрейме присутствовали дубли, использовал следующий запрос для их удаления

In [ ]:
'''
DELETE FROM team18.clinics
WHERE id IN (
    SELECT id 
    FROM (
        SELECT id, 
            ROW_NUMBER() OVER (PARTITION BY clinic_name, clinic_type, clinic_address ORDER BY id) AS rn
        FROM team18.clinics
    ) AS sub
    WHERE rn > 1
);
'''